In [1]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from IPython.display import display

## 1. Data reading

In [2]:
# load dataset

# Step 1: Extract Sheet ID and Sheet name from the link
link = "https://docs.google.com/spreadsheets/d/1vqUVWErGy5_UhfqSk3GDxN19vEUNoWDHqcfXUOdXEmQ/edit#gid=799128626"

sheet_id = link.split("/")[5]
sheet_name = link.split("gid=")[1]

# Step 2: Construct the URL
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={sheet_name}"

# Step 3: Read the URL as a pandas DataFrame
housing_df = pd.read_csv(url)

# Print the DataFrame
housing_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


## 2. Data splitting

In [3]:
# Assuming 'SalePrice' is the target variable
X_train, X_test, y_train, y_test = train_test_split(
    housing_df.drop(columns="SalePrice"),  # Features
    housing_df["SalePrice"],  # Target variable
    test_size=0.2,
    random_state=123
)

## 3. Categorical encoding - "Automated" approach (Using Pipelines)

### 3.1. Select categorical and numerical column names

In [4]:
# Select categorical and numerical column names
X_cat_columns = X_train.select_dtypes(exclude="number").columns
X_num_columns = X_train.select_dtypes(include="number").columns

### 3.2 Create the numeric pipeline

In [5]:
# Create the numeric pipeline
numeric_pipe = Pipeline([
    ('imputation', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95, svd_solver='full'))  # Use svd_solver='full' for sparse output
])

### 3.3 Create the categorical pipeline

In [6]:
# Create the categorical pipeline
categoric_pipe = Pipeline([
    ('imputation', SimpleImputer(strategy='most_frequent')),
    ('encoding', OneHotEncoder(handle_unknown='ignore'))
])

### 3.2. Using `ColumnTransformer` a pipeline with 2 branches (the `preprocessor`)

We simply tell the pipeline the following:

- One branch, called `"num_pipe"`, will apply the steps in the `numeric_pipe` to the columns named in `X_num_columns`
- The second branch, called `"cat_pipe"`, will apply the steps in the `categoric_pipe` to the columns named in `X_cat_columns`

In [7]:
# Create the column transformer for preprocessing
preprocessor = ColumnTransformer([
    ('numeric_pipe', numeric_pipe, X_num_columns),
    ('categoric_pipe', categoric_pipe, X_cat_columns)
])

### 3.3. Creating the `full_pipeline` (`preprocessor` +  Random Forest Classifier)

Pipelines are modular. The `preprocessor` we created above with the `ColumnTransformer` can become now a step in a new pipeline, that we'll call `full_piepline` and will include, as a last step, a  Decision tree model: Regression tree

In [8]:
# Create the full pipeline with the RandomForestRegressor
model = RandomForestRegressor(random_state=123)
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

### 3.4 Fit the pipeline on the training data

In [9]:
# Fit the pipeline on the modified X_train
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric_pipe',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=0.95,
                                                                       svd_solver='full'))]),
                                                  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF...
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('model', RandomForestRegressor(random_state=123))])

In [10]:
# Convert X_test to pandas DataFrame for compatibility with ColumnTransformer
X_test = pd.DataFrame(X_test, columns=X_train.columns)

### 3.5 Make predictions on the training data

In [11]:
# Make predictions on the training data
y_train_pred = full_pipeline.predict(X_train)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

# Calculate MAPE
mape = mean_absolute_percentage_error(y_train, y_train_pred)
mape_percent = mape * 100  # Convert MAPE to percentage

# Calculate R-squared
r_squared = r2_score(y_train, y_train_pred)
r_squared_percent = r_squared * 100  # Convert R-squared to percentage

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'RMSE': [rmse], 'MAPE': [mape_percent], 'R-squared': [r_squared_percent]})

# Format the metrics DataFrame
formatted_metrics_df = metrics_df.style.format({'MAPE': '{:.2f}%', 'R-squared': '{:.2f}%'})

# Display the formatted DataFrame
display(formatted_metrics_df)



,RMSE,MAPE,R-squared
0,12164.562384,3.90%,97.67%


### 3.6 Make predictions on the test data

In [12]:
# Make predictions on the test data
y_test_pred = full_pipeline.predict(X_test)

# Calculate RMSE
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

# Calculate MAPE
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
mape_percent_test = mape_test * 100  # Convert MAPE to percentage

# Calculate R-squared
r_squared_test = r2_score(y_test, y_test_pred)
r_squared_percent_test = r_squared_test * 100  # Convert R-squared to percentage

# Create a DataFrame to store the metrics for the test data
metrics_df_test = pd.DataFrame({'RMSE': [rmse_test], 'MAPE': [mape_percent_test], 'R-squared': [r_squared_percent_test]})

# Format the metrics DataFrame for the test data
formatted_metrics_df_test = metrics_df_test.style.format({'MAPE': '{:.2f}%', 'R-squared': '{:.2f}%'})

# Display the formatted DataFrame for the test data
display(formatted_metrics_df_test)

,RMSE,MAPE,R-squared
0,25681.373607,9.41%,89.33%


## 4. Train our model with GridSearch cross validation

### 4.1 Grid Search and Best Estimator

In [26]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'model__n_estimators': [10, 20],
    'model__max_depth': [None, 5],
    'model__min_samples_split': [2, 5]
}


# Create the GridSearchCV object
grid_search = GridSearchCV(full_pipeline, param_grid, cv=2, scoring='neg_mean_squared_error', n_jobs=-1)


# Fit the GridSearchCV object on the training data
grid_search.fit(X_train, y_train)

# Get the best estimator and its performance
best_estimator = grid_search.best_estimator_
best_score = np.sqrt(-grid_search.best_score_)

# Print the best estimator and its performance
print("Best Estimator:", best_estimator)
print("Best RMSE:", best_score)


Best Estimator: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric_pipe',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=0.95,
                                                                       svd_solver='full'))]),
                                                  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF...
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
     

### 4.2 Model Evaluation on Training Data

In [14]:
# Make predictions on the training data
y_train_pred = best_estimator.predict(X_train)

# Calculate RMSE
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

# Calculate MAPE
mape_train = mean_absolute_percentage_error(y_train, y_train_pred)
mape_train_percent = mape_train * 100  # Convert MAPE to percentage

# Calculate R-squared
r_squared_train = r2_score(y_train, y_train_pred)
r_squared_train_percent = r_squared_train * 100  # Convert R-squared to percentage

# Create a DataFrame to store the training metrics
metrics_train_df = pd.DataFrame({'RMSE': [rmse_train], 'MAPE': [mape_train_percent], 'R-squared': [r_squared_train_percent]})

# Format the training metrics DataFrame
formatted_metrics_train_df = metrics_train_df.style.format({'MAPE': '{:.2f}%', 'R-squared': '{:.2f}%'})

# Display the formatted training metrics DataFrame
display(formatted_metrics_train_df)


,RMSE,MAPE,R-squared
0,15941.003426,5.13%,95.99%


### 4.3 Model Evaluation on Test Data

In [15]:
# Make predictions on the test data using the best estimator
y_test_pred = best_estimator.predict(X_test)

# Calculate RMSE
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

# Calculate MAPE
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
mape_test_percent = mape_test * 100  # Convert MAPE to percentage

# Calculate R-squared
r_squared_test = r2_score(y_test, y_test_pred)
r_squared_test_percent = r_squared_test * 100  # Convert R-squared to percentage

# Create a DataFrame to store the test metrics
metrics_test_df = pd.DataFrame({'RMSE': [rmse_test], 'MAPE': [mape_test_percent], 'R-squared': [r_squared_test_percent]})

# Format the test metrics DataFrame
formatted_metrics_test_df = metrics_test_df.style.format({'MAPE': '{:.2f}%', 'R-squared': '{:.2f}%'})

# Display the formatted test metrics DataFrame
display(formatted_metrics_test_df)


,RMSE,MAPE,R-squared
0,26028.352074,9.46%,89.04%


# Competition:

In [16]:
# import Kaggle test data
url = "https://drive.google.com/file/d/1jnn7sVeWjrKyWe2DDkpbtGpM-vCmWsnW/view?usp=share_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
competition_data = pd.read_csv(path)

In [17]:
competition_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [18]:
# Print the column names of competition_data
print(competition_data.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [19]:
# Create a DataFrame for submission
my_submission = pd.DataFrame()

In [20]:
# Assign the 'Id' column from competition_data to the submission DataFrame
my_submission["Id"] = competition_data["Id"]

In [21]:
my_test_X= competition_data

In [22]:
# Predict the sale prices using the trained model and assign them to the 'SalePrice' column in the submission DataFrame
my_submission["SalePrice"] = full_pipeline.predict(my_test_X)

In [23]:
my_submission

,Id,SalePrice
0,1461,125701.79
1,1462,144547.42
2,1463,182997.70
3,1464,194129.64
4,1465,171858.82
...,...,...
1454,2915,92022.00
1455,2916,94030.23
1456,2917,192564.20
1457,2918,123044.00


In [24]:
# Save the submission DataFrame as a CSV file named 'my_submission_1.csv', without including the index column
my_submission.to_csv('my_submission_1.csv', index=False)

In [25]:
# Download the file 'my_submission_1.csv' to the local machine
from google.colab import files
files.download('my_submission_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>